In [1]:
%reload_ext autoreload
%autoreload 2
%pylab inline
%matplotlib inline

import pandas as pd
import os, random

import pickle,time

tmstmp = time.strftime("%m-%d-%Y",time.localtime())


%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


# Initialisation

## Directory paths
* TOP = path to the toplevel directory 
* TOP+/lib = path to the python source files that are required
* FIG_DIR = path to where figures are stored

In [2]:
# Set up the local source files
TOP = os.getcwd().replace('notebooks','')
LIB = TOP+'lib'
if not LIB in sys.path: sys.path.append(LIB)

DAT_DIR = TOP + '/data/'
RES_DIR = TOP + '/results/'
FIG_DIR = TOP + '/figs/'


## Load packages
Open organtox_v1 MongoDB - if this fails then check your mongodb installation

In [3]:
from organtox import *

DB = openMongo(host='localhost',auth=False,user='ishah',passwd='ishah',db='organtox_v2')
GNR = openMongo(host='localhost',auth=False,user='ishah',passwd='ishah',db='genra_dev_v5')
GNR.collection_names()

['bio_gecco_fp',
 'htpp_cr_fp',
 'toxref_gecco_fp',
 'compounds',
 'compounds_1',
 'toxcast_assays',
 'toxref_tr_fp',
 'toxcast_fp',
 'toxref_effects',
 'toxval_v9',
 'chmi_fp',
 'chms_fp',
 'httr_cr_fp',
 'httr1_gecco_fp',
 'chm_gecco_fp',
 'htpp_gecco_fp']

In [4]:
DB.collection_names()

[]

# Toxicity data

Extend the toxicity fp to include only study level hits

## See if this builds correct datasets

In [5]:
from organtox import *


DS1 = getToxDataSet('CHR:liver',MDB=GNR,
                   col_tox='toxref_tr_fp',
                   tox_pos_ds='tox_fp2.fp_pos.ds',
                   tox_neg_ds='tox_fp2.fp_neg.ds',
                   col_chm='chms_fp'
                  )


In [7]:
[[k,v.shape] for k,v in DS1.items()]

[['tox', (553, 1)],
 ['chm', (535, 2009)],
 ['bio', (503, 972)],
 ['bc', (493, 2981)]]

# Run ML Analysis

In [12]:
from organtox import *
import random
P_ALL = []

DB.ml_study_run_v1.drop()
n_np = 0
Outcomes = GNR.toxref_tr_fp.distinct('tox_fp2.fp_pos.ds')
for tox in Outcomes:
    RN0=[]
    # Get one seed for each toxicity classification
    seed = random.randint(1,1e6)      
    
    # Get the data 
    DS0 = getToxDataSet(tox,MDB=GNR,
                       col_tox='toxref_tr_fp',
                       tox_pos_ds='tox_fp2.fp_pos.ds',
                       tox_neg_ds='tox_fp2.fp_neg.ds',
                       col_chm='chms_fp'
                      )
    

    N_p,N_n = (DS0['tox'][tox]==1).sum(),(DS0['tox'][tox]==0).sum()
    
    if N_p==0 or N_n==0: continue
    print("> Tox ",tox,N_p,'+', N_n,'-',time.strftime("%H:%M",time.localtime()))
    sys.stdout.flush()
    sys.stderr.flush()
    
    for n_p in range(N_n,N_p,20):
        for dt in ['bio','chm','ct','bc','bct']:
            RN0.append(dict(tox_class=tox,descriptor_type=dt,
                            n_neg=N_n,n_pos=n_p,rand_seed=seed))
            
    DB.ml_study_run_v1.insert_many(RN0)


> Tox  CHR:adrenal gland 176 + 377 - 23:04


TypeError: documents must be a non-empty list

In [15]:
[[k,v.shape] for k,v in DS0.items()]

[['tox', (553, 1)],
 ['chm', (535, 2009)],
 ['bio', (503, 972)],
 ['bc', (493, 2981)]]

In [11]:
print DB.ml_study_run_v1.count()
#pd.DataFrame(list(DB.ml_study_run_v1.find(dict(tox_class='DEV',descriptor_type='bio'),dict(_id=0))))

450


In [12]:
DB.ml_study_lr_v1.drop()

WORK = [(w['tox_class'],w['descriptor_type'],w['n_pos'],w['n_neg'],w['rand_seed']) 
        for w in DB.ml_study_run_v1.find()]
shuffle(WORK)

print "Starting ML Analysis to assign toxicity " + time.strftime("%d/%m/%Y %H:%M",time.localtime())
print "Data sets: ", DB.ml_study_run_v1.count()


P = lb_view.map(lambda (tox_i,dt_i,np_i,nn_i,rs_i): 
                    runStudyToxML(tox_i,dt_i,np_i,nn_i,rs_i,
                                  col_tox='tox2_fp',tox_pos_ds='tox_fpp2.ds',tox_neg_ds='tox_fpn2.ds',
                                  ss_iters=10,cv_iters=2,cv_nfolds=5,
                                  n_ds_min=2,n_ds_max=22,n_ds_step=2,
                                  Col_ds=None,Col_lr=DB.ml_study_lr_v1,MDB=DB),
                    WORK)

print "Ending ML Analysis to assign activity " + time.strftime("%d/%m/%Y %H:%M",time.localtime())
#send_email(txt="Done",subj="ToxBrain Done")

Starting ML Analysis to assign toxicity 29/08/2017 19:28
Data sets:  450


CompositeError: one or more exceptions from call to method: <lambda>
[7:apply]: DataError: No numeric types to aggregate
[15:apply]: DataError: No numeric types to aggregate
[28:apply]: DataError: No numeric types to aggregate